<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/monitoring_inland_water_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring Inland Surface Water Area

In [3]:
import geemap
import ee
import shapely

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

In [8]:
y1,x1 = 27.5029,92.0958
y2,x2 = 27.5157,92.1097

bbox = (x1, y1, x2, y2)

polygon = shapely.geometry.box(*bbox, ccw=True)
geojson = eval(shapely.to_geojson(polygon))

geometry = ee.Geometry(geojson)

In [9]:
geometry

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              92.1097,
              27.5029
            ],
            [
              92.1097,
              27.5157
            ],
            [
              92.0958,
              27.5157
            ],
            [
              92.0958,
              27.5029
            ],
            [
              92.1097,
              27.5029
            ]
          ]
        ]
      }
    }
  }
})

## Download images from google earth

In [81]:
def getImageCollection(image_id, dates, geometry, cloud_cover):
    d1,d2 = dates
    ic = ee.ImageCollection(image_id)\
    .filter(ee.Filter.date(d1,d2))\
    .filter(ee.Filter.bounds(geometry))\
    .filter(ee.Filter.lt('CLOUD_COVER', cloud_cover))

    return ic

def preprocess_landsat(image:ee.image.Image, geometry: ee.Geometry) -> ee.image.Image:
    '''
    This function will choose only the required bands
    also rescale the reflectance value to original
    and calculate the indices: NDVI
    '''

    ## Select and rescale
    bands = image.select("SR_*").multiply(2.75e-05).add(-2)

    ## Calculate the NDWI
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    image = image.clip(geometry)
    return ndwi.copyProperties(image, image.propertyNames())


class AggregateImageCollection():
    def __init__(self, image_collection):
        self.image_collection = image_collection

    def extract_year(self, image, prop="system:time_start"):
        '''Extract the year from a given image'''
        year = ee.Date(image.get(prop)).get('year')
        return image.set('year', year)

    def aggregate_image_by_year(self, year):
        '''Aggregate the image collection by year'''
        yearlyImages = self.image_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        yearlyImages = yearlyImages.median()
        return yearlyImages.set('year', year)

    def aggregate_image_collection_by_year(self):
        years = self.image_collection.map(self.extract_year).aggregate_array('year').distinct().sort()
        agg_images = years.map(lambda x: self.aggregate_image_by_year(x))
        return ee.ImageCollection(agg_images)

In [82]:
cloud_cover = 5
dates = ("1999-01-01", "2025-01-01")
image_ids = ["LANDSAT/LE07/C02/T1_L2", "LANDSAT/LC08/C02/T1_L2","LANDSAT/LC09/C02/T1_L2"]

lc = None

for i, image_id in enumerate(image_ids):
    lc_images = getImageCollection(
        image_id = image_id,
        dates = dates,
        geometry = geometry,
        cloud_cover = cloud_cover
    )
    if i == 0:
        lc = lc_images
    else:
        lc = lc.merge(lc_images)

lc.size()

In [85]:
processed_lc = lc.map(lambda image:preprocess_landsat(image, geometry))

In [86]:
aggregator = AggregateImageCollection(processed_lc)
agg_lc = aggregator.aggregate_image_collection_by_year()
agg_lc.size()

## Calculate the Surface Area

## Visualize

In [87]:
years = [str(i) for i in range(1999,2023,1)]
len(years)

24

In [88]:
Map = geemap.Map()

bands = years
vis_params = {'min': -1, 'max': 1, 'palette': ['#ece7f2','#a6bddb','#2b8cbe']}

Map.add_time_slider(agg_lc, vis_params, labels=bands, time_interval=1)
Map.centerObject(geometry, zoom=11)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…